# AO3 scraping

In [1]:
#Import libraries
from bs4 import BeautifulSoup
import time
import urllib.request
import pandas as pd

In [2]:
AO3_soup = []

for x in range(1, 101):
    fanfic_page = BeautifulSoup(open(f'AO3 htmls/AO3_Page{x}.html'), 'html.parser')
    fanfic_page.prettify()
    AO3_soup.append(fanfic_page)

In [3]:
url = 'https://archiveofourown.org'

In [4]:
def getWebsiteAsSoup(url):

    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        story_html = response.read()
    
    story_soup = BeautifulSoup(story_html, 'html.parser')
    with open('story.html', 'w') as new_file:
        new_file.write(str(story_soup))
        
    return story_soup

In [5]:
stories = []
fanfics = []
relationship = []
character = []
mix_fandom = []

for x in range(len(AO3_soup)):
    for titles in AO3_soup[x].find_all('li', role = 'article'):
        fanfic = {}
        title = titles.find('a').get_text()
        fanfic['title'] = title
        
        url_tail = titles.find('a').get('href')
        link = url + url_tail
        stories.append([title, link])
        
        fandom = titles.find_all('h5', class_ = 'fandoms heading')
        fandoms = [tag.get_text().strip() for tag in fandom]
        fanfic['mixed fandom'] = fandoms
        mix_fandom.append(fandoms)
        
        relationships = titles.find_all('li', class_ = 'relationships')
        ships = [tag.get_text().strip() for tag in relationships]
        fanfic['ships'] = ships
        
        char = titles.find_all('li', class_ = 'characters')
        characters = [tag.get_text() for tag in char]
        fanfic['characters'] = (characters)
        
        try:
            hits = titles.find('dd', 'hits').get_text()
        except:
            hits = None
        fanfic['hits'] = hits
        
        try:
            kudos = titles.find('dd', 'kudos').get_text()
        except:
            kudos = None
        fanfic['kudos'] = kudos
        
        try:
            fic_soup = getWebsiteAsSoup(link)
            fic_soup.prettify()
            for info in fic_soup:
                published = info.find('dd', class_ = 'published').get_text()
                fanfic['published'] = published
                relationship.append([ships, published])
                character.append([characters, published])
        except:
            published = titles.find('p', class_ = 'datetime').get_text()
            fanfic['published'] = published
            character.append([characters, published])
            relationship.append([ships, published])
        fanfics.append(fanfic)

In [6]:
fanfics_df = pd.DataFrame.from_dict(fanfics, orient = 'columns')
fanfics_df.head()

title  \
0  Purple poison flames meet Blue fire   
1           Stuck With Babysitter Duty   
2                       Power in UA!!!   
3                  Balance and Bondeds   
4                Can't Hold Our Babies   

                                        mixed fandom  \
0  [Fandoms:\n鬼滅の刃 | Demon Slayer: Kimetsu no Yai...   
1  [Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academia...   
2  [Fandoms:\nChainsaw Man (Anime), 僕のヒーローアカデミア |...   
3  [Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academia...   
4  [Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academia...   

                                               ships  \
0           [Dabi | Todoroki Touya & Kochou Shinobu]   
1  [Dazai Osamu/Nakahara Chuuya (Bungou Stray Dogs)]   
2         [Bakugou Katsuki/Power, Power & Class 1-A]   
3  [Aizawa Shouta | Eraserhead/Yamada Hizashi | P...   
4  [Bakugou Katsuki/Midoriya Izuku/Uraraka Ochako...   

                                          characters   hits kudos    published  
0            [Dabi | Todoroki Touya, Kochou Shinobu]      0  None  02 Apr 2023  
1  [Midoriya Izuku, Dazai Osamu (Bungou Stray Dog...      0  None  02 Apr 2023  
2  [Power (Chainsaw Man), Ashido Mina, Iida Tenya...    508     7  02 Apr 2023  
3  [Midoriya Izuku, Midoriya Inko, Bakugou Katsuk...  11924   481  02 Apr 2023  
4  [Bakugou Katsuki, Uraraka Ochako, Midoriya Izuku]      0  None  02 Apr 2023

In [7]:
fanfics_df.to_csv("AO3.csv", index=False)

In [8]:
relationship.sort()

In [9]:
shipings = pd.DataFrame(relationship)
shipings.columns = ['ships', 'dates']
ships = shipings[219:]
ships = ships.explode('ships')

In [10]:
ships.to_csv("shipsviadate.csv", index=False)

In [11]:
character.sort()

In [12]:
characters = pd.DataFrame(character)
characters.columns = ['names', 'dates']
characters = characters[65:]
characters = characters.explode('names')

In [13]:
characters.to_csv('charactersviadate.csv', index = False)